# DNA Sequencing Algorithms - Homework 2
Implementing Naive and Boyer-Moore Algorithms with Comparison & Alignment Counts

In [113]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line.startswith('>'):
                genome += line.strip()
    return genome

def HammingDistance(s1, s2):
    assert len(s1) == len(s2)
    return sum([1 for a, b in zip(s1, s2) if a != b])

In [114]:
def naive_with_counts(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            num_character_comparisons += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
        num_alignments += 1
    return occurrences, num_alignments, num_character_comparisons

In [115]:
def boyer_moore_with_counts(p, p_bm, t):
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
        num_alignments += 1
    return occurrences, num_alignments, num_character_comparisons


In [116]:
def approximate_match_Index_v2(p, t, n, k):
    segment_length = int(round(len(p)/(n+1)))
    all_matches = set()
    from kmer_index import Index
    index = Index(t, k)
    index_hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = index.query(p[start:end])
        for match in matches:
            index_hits += 1
            offset = match - start
            if offset < 0 or offset + len(p) > len(t):
                continue
            if HammingDistance(p[:start], t[offset:offset+start]) + HammingDistance(p[end:], t[offset+end:offset+len(p)]) <= n:
                all_matches.add(offset)
    return list(all_matches), index_hits

In [117]:
def approximate_match_SubseqIndex(p, t, n, k, ival):
    all_matches = set()
    from kmer_subseq_index import SubseqIndex
    index = SubseqIndex(t, k, ival)
    index_hits = 0
    for i in range(n+1):
        matches = index.query(p[i:])
        for match in matches:
            index_hits += 1
            offset = match - i
            if offset < 0 or offset + len(p) > len(t):
                continue
            if HammingDistance(p, t[offset:offset+len(p)]) <= n:
                all_matches.add(offset)
    return list(all_matches), index_hits

In [118]:
def approximate_match_Index(p, t, n, k):
    segment_length = int(round(len(p)/(n+1)))
    all_matches = set()
    from kmer_index import Index
    index = Index(t, k)
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = index.query(p[start:end])
        for match in matches:
            offset = match - start
            if offset < 0 or offset + len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if p[j] != t[offset+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if p[j] != t[offset+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(offset)
    return list(all_matches)

In [119]:
from bm_preproc import BoyerMoore

p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
t = readGenome("chr1.GRCh38.excerpt.fasta")

# Answer to Q1
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print("Naive - Alignments Tried:", num_alignments)

# Answer to Q2
print("Naive - Character Comparisons:", num_character_comparisons)

# Answer to Q3

p_bm = BoyerMoore(p, "ACGT")
bm_occurrences, bm_num_alignments, bm_num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print("Boyer-Moore - Alignments Tried:", bm_num_alignments)


# Answer to Q4 & Q5
p_4 = "GGCGCGGTGGCTCACGCCTGTAAT"
approx_occurrences, index_hits = approximate_match_Index_v2(p_4, t, 2, 8)
print("Approximate Matches (<=2 mismatches):", len(approx_occurrences))
print("Total Index Hits:", index_hits)

# Q6
occurrences_subseq, index_hits_subseq = approximate_match_SubseqIndex(p_4, t, 2, 8, 3)
print("SubseqIndex - Total Index Hits:", index_hits_subseq)


Naive - Alignments Tried: 799954
Naive - Character Comparisons: 984143
Boyer-Moore - Alignments Tried: 127974
Approximate Matches (<=2 mismatches): 19
Total Index Hits: 90
SubseqIndex - Total Index Hits: 79
